In [1]:
import pandas as pd
import numpy as np

In [411]:
class MLP:
    def __init__(self, num_inputs=4, num_hidden=[5,6], num_output=2):
        
        
        #### initialize layers ###
        
        for inp in num_hidden:
            inp+=1
        
        self.num_inputs = num_inputs +1
        self.num_hidden = num_hidden
        self.num_output = num_output
        
        self.layers = [num_inputs] + num_hidden + [num_output]
        
        
        
        ### initialize weights ###
        
        self.weights = []
        for i in range(0,len(self.layers)-1): #no weights for input layer
            weight = np.random.rand(self.layers[i], self.layers[i+1])
            self.weights.append(weight)
        
        
        
        ### initialize Activations ###
        
        self.activations = []
        for i in range(len(self.layers)):
            temp_zeros = np.zeros(self.layers[i])
            self.activations.append(temp_zeros)
            
        
        ### initialize Derivatives ###
        
        self.derivatives = []
        for i in range(len(self.layers)-1):
            temp_zeros = np.zeros((self.layers[i], self.layers[i+1]))
            self.derivatives.append(temp_zeros)
        
        
    def sigmoid(self, x):
        return 1/(1+np.exp(-1*x))
    
    def sigmoid_derivative(self,sigx):
        return sigx*(1.0-sigx)
    
    def Relu(self, x):
        return np.maximum(0,x)
    
    def Relu_derivative(self,relux):
        res = []
        for ele in relux:
            res.append(0) if ele==0 else res.append(1)
        return res
    
    def forward_propagation(self,inputs, activation='Relu'):
        activations = inputs
        self.activations[0] = inputs
        
        #iterate throuugh wi
        for i,weight in enumerate(self.weights):
            
            #calculate each layer input (hi)
            net_inputs = np.dot(activations, weight)
            
            if i == len(self.weights)-1:
                break
            if activation == 'Sigmoid':
                #Calculate the activations (ai)
                activations = self.sigmoid(net_inputs)

            elif activation == 'Relu':
                activations = self.Relu(net_inputs)
  
            self.activations[i+1] = activations
            
        activations = self.sigmoid(net_inputs)   
        self.activations[len(self.weights)] = activations
        return activations
    
    def back_propagation(self, error, activation = 'Relu'):
        
        for i in reversed(range(len(self.derivatives))):
            ## get Sigmoid(h[i+1]) = activations[i+1]
            activations = self.activations[i+1] 
            
            if i == len(self.derivatives)-1 or activation == 'Sigmoid': 
                ## delta = (a[i+1] - y) * sigmoid'(h[i+1])
                delta = error * self.sigmoid_derivative(activations)
            else:
                ## delta = (a[i+1] - y) * sigmoid'(h[i+1])
                delta = error * self.Relu_derivative(activations)

            ## calculate error derivative of current layer ## (delta E/ delta Wi) 

            current_act = self.activations[i]
            
            #transform current_act into 3x1 matrix (layers[i]x1)
            current_act = current_act.reshape(current_act.shape[0], -1)  #-1 tells to automatically calculate the value of the column
            
            #transform delta into 1X2 matrix (1xlayers[i+1])

            delta = delta.reshape(1,len(delta))
            
            ## (delta E/ delta Wi) ##
            self.derivatives[i] = np.dot(current_act, delta) #(layers[i]xlayers[i+1])
            
            ## calculate new error ##
            error = np.dot(delta,self.weights[i].T).flatten()
            

        return error
    
    
    def gradient_decent(self, learning_rate = 0.1):
        
        for i in range(len(self.weights)):
            weight = self.weights[i]
            derivative = self.derivatives[i]

            weight = weight + learning_rate * derivative

            self.weights[i] = weight
    
    def train(self,inputs, targets, epochs = 50, learning_rate = 0.2, verbose = False, activation = 'Relu'):
        for inp in inputs:
            np.append(inp,1)
        for i in range(0,epochs):   
            sum_error = 0
            if verbose:
                print("\n-------------Epoch: {}----------------\n".format(i+1))
            
            for (input, target) in zip(inputs,targets):
                
                #forward prop
                output = self.forward_propagation(input, activation = activation)

                #calculate error
                error = target - output
                #print("tar-out-err: ",target, self.sign(output), error)
                #sum_error +=self.mse(target, output)
                sum_error += abs(error)
                
                #back Prop
                error = self.back_propagation(error, activation = 'Relu')

                
                #gradient descent
                self.gradient_decent(learning_rate)
            if verbose:
                print("-------------ERROR: {}----------------\n\n".format((sum_error.round(3))/(inputs.shape[0])))            
    
    def sign(self,x):
        
        return 0 if x<0 else 1
    
    def mse(self,target, output):
        sum = 0
        for i in range(len(target)):
            sum+= (target[i]-output[i])**2
        return sum/len(target)

    def classify_predict(self,input):
        output = forward_propagation(input)
        pred = np.where(output == max(output))[0]
        return pred
    
    def regression_predict(self,input):
        output=self.forward_propagation(input)
        return output

In [389]:
mlp = MLP(2,[],1)

In [390]:
inputs = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[0],[0],[1]])

In [391]:
mlp.train(inputs,target, 1000, learning_rate = 2, verbose = True, activation = 'Sigmoid')


-------------Epoch: 1----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.63812398]
tar-out-err:  [0] 1 [-0.67616432]
tar-out-err:  [1] 1 [0.32902388]
-------------ERROR: [0.53575]----------------



-------------Epoch: 2----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.60295497]
tar-out-err:  [0] 1 [-0.64230005]
tar-out-err:  [1] 1 [0.39667925]
-------------ERROR: [0.5355]----------------



-------------Epoch: 3----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.57907274]
tar-out-err:  [0] 1 [-0.61776999]
tar-out-err:  [1] 1 [0.44398159]
-------------ERROR: [0.53525]----------------



-------------Epoch: 4----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.56362257]
tar-out-err:  [0] 1 [-0.60050018]
tar-out-err:  [1] 1 [0.4755049]
-------------ERROR: [0.535]----------------



-------------Epoch: 5----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.55374426]
tar-out-err:  [0] 1 [-0.58822001]
tar-o

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 406----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 407----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 408----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 409----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]


tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 732----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 733----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 734----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.53375]----------------



-------------Epoch: 735----------------

tar-out-err:  [0] 1 [-0.5]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [0] 1 [-0.54492217]
tar-out-err:  [1] 1 [0.54492217]
-------------ERROR: [0.5337

In [392]:
mlp.regression_predict([0,0])

array([0.5])

In [393]:
mlp.regression_predict([1,0])

array([0.54492217])

In [394]:
mlp.regression_predict([0,1])

array([0.54492217])

In [395]:
mlp.regression_predict([1,1])

array([0.58912492])

## Mobile Phone Sales

In [412]:
df = pd.read_csv(r"C:\Users\rvikr\CEG ML Lab\train.csv")
df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [413]:
df = df[df['label'].isin([1,0])]

In [414]:
inputs = df.drop(['label'], axis=1)
target = df['label']

In [415]:
inputs.shape

(8816, 784)

In [416]:
def one_hot_encode(X):
    unique_values = list(set(X))
    unique_values.sort()
    res = [[1 if num == value else 0 for value in unique_values] for num in X]
    return res

In [417]:
target = one_hot_encode(target)

In [418]:
inputs.shape

(8816, 784)

In [419]:
targets = np.array(target)

In [420]:
targets.shape

(8816, 2)

In [421]:
inputs = inputs.values

In [422]:
inputs

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [426]:
mlp = MLP(784,[64, 32, 16, 8],2)

In [427]:
mlp.train(inputs,targets,5, 10,True)


-------------Epoch: 1----------------

-------------ERROR: [0.53130672 0.46869328]----------------



-------------Epoch: 2----------------

-------------ERROR: [0.53130672 0.46869328]----------------



-------------Epoch: 3----------------

-------------ERROR: [0.53130672 0.46869328]----------------



-------------Epoch: 4----------------

-------------ERROR: [0.53130672 0.46869328]----------------



-------------Epoch: 5----------------

-------------ERROR: [0.53130672 0.46869328]----------------


